## Data Cleaning for Food Environment Project

In [ ]:
# Import libraries
import pandas as pd
import seaborn as sns # for visualizations

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define directory where data is located
data_dir =  "/content/drive/My Drive/DS 5100 Food Environment Project/FoodEnvironmentAtlas/"

# Display files located there
!ls  "/content/drive/My Drive/DS 5100 Food Environment Project/FoodEnvironmentAtlas"

ReadMeFile2020.txt	   SupplementalDataCounty.csv  VariableList.gsheet
StateAndCountyData.csv	   SupplementalDataState.csv
StateAndCountyData.gsheet  VariableList.csv


#### State and County Data

In [ ]:
# Read in state and county data
df = pd.read_csv(data_dir + 'StateAndCountyData.csv')
df.head()

FIPS State   County          Variable_Code         Value
0  1001    AL  Autauga          LACCESS_POP10  18428.439690
1  1001    AL  Autauga          LACCESS_POP15  17496.693040
2  1001    AL  Autauga  PCH_LACCESS_POP_10_15     -5.056026
3  1001    AL  Autauga      PCT_LACCESS_POP10     33.769657
4  1001    AL  Autauga      PCT_LACCESS_POP15     32.062255

In [ ]:
df.State.nunique()

101

In [ ]:
df.State.value_counts()

TX     66315
GA     41543
VA     33155
KY     31738
MO     30332
       ...  
 CT       72
 RI       45
 HI       45
 DE       27
 DC        9
Name: State, Length: 101, dtype: int64

In [ ]:
# Cleaning state to remove spaces
def clean_state_column(df):
  df['State'] = df['State'].str.strip()
  return df

df = clean_state_column(df)

In [ ]:
df.State.value_counts()

TX    68601
GA    42974
VA    34352
KY    32818
MO    31367
KS    28008
NC    27743
IL    27675
IA    26896
TN    26108
IN    24878
NE    24542
OH    24431
MN    23700
MI    23074
MS    21366
OK    21021
AR    20266
WI    20121
PA    18873
AL    18504
FL    18415
LA    17385
NY    17376
SD    17354
CO    17137
CA    16261
WV    15060
MT    14889
ND    13873
SC    12741
ID    11957
WA    10897
OR     9945
NM     9019
UT     7926
MD     6698
AK     6455
WY     6107
NJ     5874
ME     4555
NV     4460
AZ     4108
VT     3977
MA     3922
NH     2861
CT     2322
RI     1426
HI     1351
DE      878
DC      263
Name: State, dtype: int64

In [ ]:
# Look at how many counties we have
df.County.nunique()

3708

In [ ]:
df.County.value_counts()

Washington           8325
Jefferson            6904
Franklin             6847
Jackson              6351
Lincoln              6257
                     ... 
Power County            9
Mahnomen County         9
Hot Spring County       9
Morehouse Parish        9
Raleigh County          9
Name: County, Length: 3708, dtype: int64

Web Scrape State/County Lookup Table

In [ ]:
# For county name cleaning will web scrape to get lookup table
# Due to different names country vs census area didn't think we could clean easily

# Web scrape lookup table from here: https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
# Define header
headers = {'user-agent': 'UVA Project (pkx2ec@virginia.edu) (Language=Python 3.8.2; Platform=Macintosh; Intel Mac OS X 11_5_2)'} 

# Specify URL
URL = 'https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697'

# Access HTML content 
r = requests.get(URL, headers= headers)

# Parse HTML content
soup = BeautifulSoup(r.content, 'html5lib')




In [ ]:
soup.prettify()

'<!DOCTYPE html>\n<html lang="en" xmlns="http://www.w3.org/1999/xhtml">\n <head>\n  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>\n  <title>\n   County FIPS Codes | NRCS\n  </title>\n  <script type="text/javascript">\n   // Instructions: please embed this snippet directly into every page in your website template.\n    // For optimal performance, this must be embedded directly into the template, not referenced\n    // as an external file.\n\n    // Answers Cloud Services Embed Script v1.02\n    // DO NOT MODIFY BELOW THIS LINE *****************************************\n    ;(function (g) {\n        var d = document, i, am = d.createElement(\'script\'), h = d.head || d.getElementsByTagName("head")[0],\n                aex = {\n                    "src": "//gateway.answerscloud.com/nrcs-usda-gov/production/gateway.min.js",\n                    "type": "text/javascript",\n                    "async": "true",\n                    "data-vendor": "acs",\n                    

In [ ]:
# Find data of interest
table = soup.find('div', attrs = {'class':'centerColImg'}) 

In [ ]:
# Create empty dataframe to save FIPS info
fips_table = pd.DataFrame(columns = ['FIPS', 'County', 'State'])
fips_table

Empty DataFrame
Columns: [FIPS, County, State]
Index: []

In [ ]:
# Add each FIPS code to dataframe
first_row = True
for row in table.findAll('tr'):
  # First row doesn't have the info we want so skip it
  if first_row:
    first_row = False
  else:
    row_entries = row.findAll('td')
    row_text = [i.text for i in row_entries]
    fips_table.loc[len(fips_table)] = row_text

In [ ]:
# Check first 5 rows of our FIPS table
fips_table.head()

FIPS   County State
0  01001  Autauga    AL
1  01003  Baldwin    AL
2  01005  Barbour    AL
3  01007     Bibb    AL
4  01009   Blount    AL

In [ ]:
df.head()

FIPS State   County          Variable_Code         Value
0  1001    AL  Autauga          LACCESS_POP10  18428.439690
1  1001    AL  Autauga          LACCESS_POP15  17496.693040
2  1001    AL  Autauga  PCH_LACCESS_POP_10_15     -5.056026
3  1001    AL  Autauga      PCT_LACCESS_POP10     33.769657
4  1001    AL  Autauga      PCT_LACCESS_POP15     32.062255

In [ ]:
df.dtypes

FIPS               int64
State             object
County            object
Variable_Code     object
Value            float64
dtype: object

In [ ]:
fips_table.dtypes

FIPS      object
County    object
State     object
dtype: object

In [ ]:
# Change type of fips table to int so we join it
fips_table['FIPS'] = fips_table['FIPS'].astype(int)

In [ ]:
fips_table.head()

FIPS   County State
0  1001  Autauga    AL
1  1003  Baldwin    AL
2  1005  Barbour    AL
3  1007     Bibb    AL
4  1009   Blount    AL

In [ ]:
original_fips = df.FIPS.unique()
new_fips = fips_table.FIPS.unique()

for i in original_fips:
  if i not in new_fips:
    fips_table.loc[len(fips_table.index)] = df.loc[df['FIPS']==i,['FIPS', 'County', 'State']].iloc[0]

In [ ]:
# Drop original state and county columns 
# Otherwise we'll have 2 columns with the same name
df.drop(['State', 'County'], axis=1, inplace = True)

In [ ]:
# Deal with the leading zero in FIPS column
# Nevermind, looks like it handled this when we converted to int

In [ ]:
# Check shape of original dataframe so we know if we drop data
df.shape

(852810, 3)

In [ ]:
df.FIPS.nunique()

3196

In [ ]:
fips_table.FIPS.nunique()

3288

In [ ]:
# Join together fips lookup table
# Left join so we don't lose any data from our original table
df_new = df.merge(fips_table, on = ['FIPS'], how = 'left')


In [ ]:
df_new.head()

FIPS          Variable_Code         Value   County State
0  1001          LACCESS_POP10  18428.439690  Autauga    AL
1  1001          LACCESS_POP15  17496.693040  Autauga    AL
2  1001  PCH_LACCESS_POP_10_15     -5.056026  Autauga    AL
3  1001      PCT_LACCESS_POP10     33.769657  Autauga    AL
4  1001      PCT_LACCESS_POP15     32.062255  Autauga    AL

In [ ]:
df_new.shape

(852810, 5)

In [ ]:
# Missing values for county where we didn't have match
df_new.County.isnull().sum()

0

In [ ]:
df_new[df_new['County'].isnull()]

Empty DataFrame
Columns: [FIPS, Variable_Code, Value, County, State]
Index: []

In [ ]:
df_new.head()

FIPS          Variable_Code         Value   County State
0  1001          LACCESS_POP10  18428.439690  Autauga    AL
1  1001          LACCESS_POP15  17496.693040  Autauga    AL
2  1001  PCH_LACCESS_POP_10_15     -5.056026  Autauga    AL
3  1001      PCT_LACCESS_POP10     33.769657  Autauga    AL
4  1001      PCT_LACCESS_POP15     32.062255  Autauga    AL

Reformat data

In [ ]:
# Re-format so variables are across the columns not adding rows in the "Variable Code" column
pivot = pd.pivot_table(df_new, index = ['FIPS', 'State', 'County'], columns = 'Variable_Code', values = 'Value')
pivot.reset_index(inplace=True)
pivot # format looks better for analysis, but lots of NaNs...


Variable_Code   FIPS State  ... WIC_PART_2017  WIC_PART_2018
0                  1    AL  ...  1.239926e+05   1.206048e+05
1                  2    AK  ...  1.818817e+04   1.709242e+04
2                  4    AZ  ...  1.535101e+05   1.495133e+05
3                  5    AR  ...  7.651850e+04   7.360675e+04
4                  6    CA  ...  1.080241e+06   1.009492e+06
...              ...   ...  ...           ...            ...
3191           56037    WY  ...           NaN            NaN
3192           56039    WY  ...           NaN            NaN
3193           56041    WY  ...           NaN            NaN
3194           56043    WY  ...           NaN            NaN
3195           56045    WY  ...           NaN            NaN

[3196 rows x 335 columns]

In [ ]:
# Look at missing data
percent_missing = pivot.isnull().sum() * 100 / len(pivot)
percent_missing
# Looks to be a high percentage of missing data in some columns
# From inspection though looks like some are due to "Total" county (98.40% missing)
# I.e. state-wide variables so they are null for each county
# Definitely some data cleaning to be done

Variable_Code
2010_Census_Population     1.689612
AGRITRSM_OPS07             3.629537
AGRITRSM_OPS12             3.629537
AGRITRSM_RCT07            38.235294
AGRITRSM_RCT12            33.698373
                            ...    
WIC_PART_2014             98.404255
WIC_PART_2015             98.404255
WIC_PART_2016             98.404255
WIC_PART_2017             98.404255
WIC_PART_2018             98.404255
Length: 332, dtype: float64

Split state and county data into separate tables

In [ ]:
df_state = pivot[pivot.FIPS < 60] # (FIPS 1-56 are state-level)

In [ ]:
df_county = pivot[pivot.FIPS >= 60]

In [ ]:
# Look at missing data for county
percent_missing = df_county.isnull().sum() * 100 / len(df_county)
percent_missing # some with 100% missing must have been those state total rows

Variable_Code
FIPS                        0.000000
State                       0.000000
County                      0.000000
2010_Census_Population      0.095390
AGRITRSM_OPS07              2.066773
                             ...    
WIC_PART_2014             100.000000
WIC_PART_2015             100.000000
WIC_PART_2016             100.000000
WIC_PART_2017             100.000000
WIC_PART_2018             100.000000
Length: 335, dtype: float64

In [ ]:
# Drop columns where we have all nas 
df_state = df_state.dropna(axis=1, how='all')
df_county = df_county.dropna(axis=1, how='all')

In [ ]:
df_county.isnull().sum()

Variable_Code
FIPS                        0
State                       0
County                      0
2010_Census_Population      3
AGRITRSM_OPS07             65
                         ... 
VLFOODSEC_15_17             2
WICS11                    134
WICS16                    160
WICSPTH11                 137
WICSPTH16                 163
Length: 293, dtype: int64

Save cleaned data back to google drive


In [ ]:
# Define directory where to save cleaned data
data_dest =  "/content/drive/My Drive/DS 5100 Food Environment Project/Data_Cleaned/"

df_state.to_csv(data_dest + 'State_Level_Data.csv', index=False)
df_county.to_csv(data_dest + 'County_Level_Data.csv', index=False)

Preliminary exploration....

In [ ]:
# Just picking a few columns to look at for now to get an idea....
cols = ['PCT_LACCESS_HHNV15', 'GROCPTH16', 'FFRPTH16', 'PCT_SNAP17', 
        'FOODINSEC_15_17', 'PCT_OBESE_ADULTS17', 'POVRATE15']
pivot_subset = pivot[cols]
# Column descriptions
# 'PCT_LACCESS_HHNV15': Households, no car & low access to store (%), 2015
# 'GROCPTH16': Grocery stores/1,000 pop, 2016
# 'FFRPTH16': Fast-food restaurants/1,000 pop, 2016
# 'PCT_SNAP17': SNAP participants (% pop), 2017*
# 'FOODINSEC_15_17': Household food insecurity (%, three-year average), 2015-17*
# 'PCT_OBESE_ADULTS17': Adult obesity rate, 2017*
# 'POVRATE15': Poverty rate, 2015

In [ ]:
# Check correlations
pivot_subset.corr()

Variable_Code       PCT_LACCESS_HHNV15  ...  POVRATE15
Variable_Code                           ...           
PCT_LACCESS_HHNV15            1.000000  ...   0.452562
GROCPTH16                     0.120120  ...  -0.060947
FFRPTH16                     -0.205692  ...  -0.062722
PCT_SNAP17                    0.191959  ...   0.432630
FOODINSEC_15_17               0.039258  ...   0.166379
PCT_OBESE_ADULTS17            0.184312  ...   0.221266
POVRATE15                     0.452562  ...   1.000000

[7 rows x 7 columns]

Some simple visualizations

In [ ]:
# Poverty rate vs adult obesity rate
sns.scatterplot(x='POVRATE15', y='PCT_OBESE_ADULTS17',  data=pivot_subset)

In [ ]:
# Food insecurity vs adult obesity rate
sns.scatterplot(x='FOODINSEC_15_17', y='PCT_OBESE_ADULTS17',  data=pivot_subset)

In [ ]:
# Poverty rate vs low access to store
sns.scatterplot(x='POVRATE15', y='PCT_LACCESS_HHNV15',  data=pivot_subset)

In [1]:
# Test Git